In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import urllib.request as request

from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent

In [2]:
options = Options()
ua = UserAgent(verify_ssl=False)
userAgent = ua.random
print(userAgent)

Error occurred during loading data. Trying to use cache server https://fake-useragent.herokuapp.com/browsers/0.1.11
Traceback (most recent call last):
  File "/Users/tj/opt/anaconda3/lib/python3.9/site-packages/fake_useragent/utils.py", line 154, in load
    for item in get_browsers(verify_ssl=verify_ssl):
  File "/Users/tj/opt/anaconda3/lib/python3.9/site-packages/fake_useragent/utils.py", line 99, in get_browsers
    html = html.split('<table class="w3-table-all notranslate">')[1]
IndexError: list index out of range


Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_6_6; de-de) AppleWebKit/533.20.25 (KHTML, like Gecko) Version/5.0.4 Safari/533.20.27


In [3]:
options.add_argument(f'user-agent={userAgent}')
# chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.get('https://www.melon.com/mymusic/dj/mymusicdjplaylistview_inform.htm?plylstSeq=458511975')

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - About to download new driver from https://chromedriver.storage.googleapis.com/103.0.5060.134/chromedriver_mac64.zip
[WDM] - Driver has been saved in cache [/Users/tj/.wdm/drivers/chromedriver/mac64/103.0.5060.134]


In [4]:
page_source = driver.page_source
soup = BeautifulSoup(page_source, "html.parser")

In [5]:
import time

total_lists = []

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.get('https://www.melon.com/mymusic/dj/mymusicdjplaylistview_inform.htm?plylstSeq=458511975')

time.sleep(1)

page_source = driver.page_source
soup = BeautifulSoup(page_source, "html.parser")

time.sleep(1)

songIds = soup.select("#frm > div > table > tbody td:nth-child(4) > div > a")
total_lists = total_lists + songIds

time.sleep(1)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [/Users/tj/.wdm/drivers/chromedriver/mac64/103.0.5060.134/chromedriver] found in cache


In [6]:
test = []
for i in range(len(total_lists)):
    test.append(total_lists[i]['href'])

In [7]:
test1 = []
for i in range(len(test)):
    test1.append(str(test[i]).split('.')[2].split("(")[1].split(")")[0])

In [8]:
test2 = []
for i in range(len(test1)):
    test2.append(str(test1[i]).strip("''"))

In [9]:
test2

['8193669',
 '4335500',
 '31754430',
 '1975152',
 '2678080',
 '1934285',
 '30554140',
 '497095',
 '30378161',
 '7886240',
 '4268560',
 '8028725',
 '4412480']

In [10]:
# 가수명, 노래제목, 발매일, 가사, 좋아요 수 리스트 변수 생성

singer_list = []
singName_list = []
date_list = []
genre_list = []
lyrics_list = []
like_list = []

In [11]:
# for문으로 가수명, 노래제목, 발매일, 가사, 좋아요 수 리스트 변수에 내용 담고 저장하기

chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

for songId in test2:
    
    driver.get(f'https://www.melon.com/song/detail.htm?songId={songId}')

    page_source = driver.page_source
    soup = BeautifulSoup(page_source, "html.parser")

    time.sleep(2)

    try:
    
        # 가수명 가져오기
        artist_name = soup.select_one('#downloadfrm > div > div > div.entry > div.info > div.artist > a > span:nth-child(1) ')
        singer_list.append(artist_name.string)

        # 노래제목 가져오기
        song_name = soup.select_one("#downloadfrm > div > div > div.entry > div.info > div.song_name ")
        singName_list.append(song_name.text.strip().split('\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t')[1])

        # 발매일 가져오기
        date = soup.select_one('#downloadfrm > div > div > div.entry > div.meta > dl > dd:nth-child(4)')
        date_list.append(date.string)

        # 장르 가져오기
        genre = soup.select_one('#downloadfrm > div > div > div.entry > div.meta > dl > dd:nth-child(6)')
        genre_list.append(genre.string)

        # 가사 가져오기
        lyric = str(soup.select_one('#d_video_summary')).replace('<br/>', ' ').replace(
            '<div class="lyric" id="d_video_summary"><!-- height:auto; 로 변경시, 확장됨 -->', '').replace('</div>', '').replace('\t', '').replace('\n', '')
        lyric = lyric.replace('  ', ' ')
        lyrics_list.append(lyric)

        # 좋아요수 가져오기
        like = soup.select_one("#d_like_count")
        like_list.append(like.text)

        time.sleep(2)
    except:
        singer_list.append(0)
        singName_list.append(0)
        date_list.append(0)
        lyrics_list.append(0)
        like_list.append(0)

        time.sleep(1)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [/Users/tj/.wdm/drivers/chromedriver/mac64/103.0.5060.134/chromedriver] found in cache


In [12]:
ballad = {'가수명':singer_list, '노래제목':singName_list, '발매일':date_list, '장르':genre_list, '가사':lyrics_list, '좋아요수':like_list}

In [13]:
import pandas as pd

In [14]:
ballad_calm= pd.DataFrame(ballad)
ballad_calm.head()

,가수명,노래제목,발매일,장르,가사,좋아요수
0,백아연,할 말,2016.05.24,발라드,어려운 말을 꺼내요 그 동안 그댈 기다리는 것 말고는할 수 있는 게 없던 나죠 이제...,"9,840"
1,딕펑스 (DICKPUNKS),바래져,2013.11.27,록/메탈,한장 두장씩 책을 넘기다 무심코 머문 그 페이지에는 니가 남겨둔 짧은 편지가 남겨져...,"4,159"
2,손동운,Decrescendo,2019.04.22,발라드,내게 주던 사랑이갈수록 익숙해져편하게 생각했나 봐날 떠나 가벼워진 사랑은이제야 겨우...,"3,906"
3,홍민정,아마도 그건,2008.11.28,"발라드, 국내영화",아마도 그건 사랑이었을거야희미하게 떠오르는 기억이이제야 그 마음을 알아버렸네그대 눈...,"27,493"
4,에피톤 프로젝트,한숨이 늘었어 (Duet With 이진우),2010.05.12,"발라드, 인디음악",요즘 재밌다는 영화유행하는 것들 일부러 찾곤 해조금 웃을 수 있어서잠깐 잊을 수 있...,"3,824"


In [15]:
ballad_calm.장르.unique()

array(['발라드', '록/메탈', '발라드, 국내영화', '발라드, 인디음악', '인디음악, 록/메탈', '발라드, 록/메탈'],
      dtype=object)

In [16]:
list = ['발라드, 국내영화','발라드, 인디음악','발라드','발라드,록/메탈']
ballad_calm = ballad_calm[ballad_calm['장르'].isin(list)]

In [17]:
ballad_calm.reset_index(drop=True, inplace=True)

In [19]:
ballad_calm.drop(columns='장르',inplace=True)

In [20]:
ballad_calm

,가수명,노래제목,발매일,가사,좋아요수
0,백아연,할 말,2016.05.24,어려운 말을 꺼내요 그 동안 그댈 기다리는 것 말고는할 수 있는 게 없던 나죠 이제...,"9,840"
1,손동운,Decrescendo,2019.04.22,내게 주던 사랑이갈수록 익숙해져편하게 생각했나 봐날 떠나 가벼워진 사랑은이제야 겨우...,"3,906"
2,홍민정,아마도 그건,2008.11.28,아마도 그건 사랑이었을거야희미하게 떠오르는 기억이이제야 그 마음을 알아버렸네그대 눈...,"27,493"
3,에피톤 프로젝트,한숨이 늘었어 (Duet With 이진우),2010.05.12,요즘 재밌다는 영화유행하는 것들 일부러 찾곤 해조금 웃을 수 있어서잠깐 잊을 수 있...,"3,824"
4,진영 (GOT7),그날,2017.07.31,누가 말 걸어 주지 않으면아무 말 없이나도 모르게 찾게 되는 건한가지 너고요했던 이...,"10,594"
5,장나라,그게 정말이니?,2003.12.03,후회는 없을거라고 그만큼 사랑했다고지금 흔들리는 목소리로이별을 얘기하는거니사랑한 순...,"9,600"
6,아이유,마침표,2017.04.21,알고 있었어 무슨 말인지무슨 마음인지 다 알아하루 더 딱 하루만 더미루고 싶었어그래...,"67,534"
7,스윗소로우 (SWEET SORROW),서울은 비,2015.10.15,오늘 따라 내리는 이 비는좀처럼 그치질 않네요혹시 그대 있는 곳에도지금처럼 비가 내...,"3,226"
8,지코 (ZICO),사랑이었다 (Feat. 루나 of f(x)),2016.01.25,알 수 없다눈 뜨고 꿈꾸는 기분일까괜히 가슴이 소란스러워지고그리움도 경험해본다내게 ...,"121,044"


In [21]:
ballad_calm['좋아요수'] = ballad_calm['좋아요수'].str.replace(',','')

In [22]:
ballad_calm['발매일'] = ballad_calm['발매일'].str.replace('.', '-')

/var/folders/51/xmc_l6t13wv1nc5z6qmyg58r0000gn/T/ipykernel_2477/2831921886.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  ballad_calm['발매일'] = ballad_calm['발매일'].str.replace('.', '-')


In [23]:
ballad_calm['발매일'] = pd.to_datetime(ballad_calm['발매일'])

In [24]:
len(ballad_calm)

9

In [25]:
ballad_calm.to_csv('../Data/ballad_calm3.csv', index=False)

In [26]:
pd.read_csv("../Data/ballad_calm3.csv")

,가수명,노래제목,발매일,가사,좋아요수
0,백아연,할 말,2016-05-24,어려운 말을 꺼내요 그 동안 그댈 기다리는 것 말고는할 수 있는 게 없던 나죠 이제...,9840
1,손동운,Decrescendo,2019-04-22,내게 주던 사랑이갈수록 익숙해져편하게 생각했나 봐날 떠나 가벼워진 사랑은이제야 겨우...,3906
2,홍민정,아마도 그건,2008-11-28,아마도 그건 사랑이었을거야희미하게 떠오르는 기억이이제야 그 마음을 알아버렸네그대 눈...,27493
3,에피톤 프로젝트,한숨이 늘었어 (Duet With 이진우),2010-05-12,요즘 재밌다는 영화유행하는 것들 일부러 찾곤 해조금 웃을 수 있어서잠깐 잊을 수 있...,3824
4,진영 (GOT7),그날,2017-07-31,누가 말 걸어 주지 않으면아무 말 없이나도 모르게 찾게 되는 건한가지 너고요했던 이...,10594
5,장나라,그게 정말이니?,2003-12-03,후회는 없을거라고 그만큼 사랑했다고지금 흔들리는 목소리로이별을 얘기하는거니사랑한 순...,9600
6,아이유,마침표,2017-04-21,알고 있었어 무슨 말인지무슨 마음인지 다 알아하루 더 딱 하루만 더미루고 싶었어그래...,67534
7,스윗소로우 (SWEET SORROW),서울은 비,2015-10-15,오늘 따라 내리는 이 비는좀처럼 그치질 않네요혹시 그대 있는 곳에도지금처럼 비가 내...,3226
8,지코 (ZICO),사랑이었다 (Feat. 루나 of f(x)),2016-01-25,알 수 없다눈 뜨고 꿈꾸는 기분일까괜히 가슴이 소란스러워지고그리움도 경험해본다내게 ...,121044
